# Importation des librairies

In [2]:
#Importation des librairies 
import pandas as pd 
import os
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
import category_encoders as ce
import joblib

In [3]:
os.chdir("/Users/pierrebourbon/Documents/PRO/Data/Master_SISE/Programmation_Python/Projet")

# Chargement des données 

In [4]:
df = pd.read_excel("extract-dpe.xlsx")

In [100]:
#Choix des variables explicatives

var_explicatives = ["Etiquette_DPE", 
                    "Type_bâtiment", 
                    "Année_construction", 
                    "Classe_inertie_bâtiment", 
                    "Hauteur_sous-plafond", 
                    "Surface_habitable_logement",  
                    "Type_énergie_principale_chauffage", 
                    "Isolation_toiture_(0/1)", "Code_INSEE_(BAN)"]

target = ["Conso_5_usages_é_finale"]

In [101]:
#Création d'un data frame avec les variables explicatives et la target
df_dpe = df[var_explicatives+target]

# Suppression des outliers, NA de la target et Type_batiment == immeuble

In [102]:
#Supression des outliers avec la méthode des quartiles [q1 - 1.5 * IQR , q3 + 1.5 * IQR]

#Sélection des vars quanti
df_quanti = df_dpe.select_dtypes(include=['number'])

#Calcul du IQR pour chaque colonne 
Q1 = df_quanti.quantile(0.025)
Q3 = df_quanti.quantile(0.975)
IQR = Q3-Q1

#Filtrage des données sans outliers 
df_dpe_filtered = df_dpe[~((df_quanti < (Q1 - 1.5 * IQR)) | (df_quanti > (Q3 + 1.5 * IQR))).any(axis=1)]

df_dpe_filtered.describe()

,Année_construction,Hauteur_sous-plafond,Surface_habitable_logement,Isolation_toiture_(0/1),Conso_5_usages_é_finale
count,186204.000000,313695.000000,311641.000000,186839.000000,313687.000000
mean,1975.358322,2.589259,65.811322,0.355156,10499.919965
std,28.364826,0.235743,34.695060,0.478562,9562.245845
min,1731.000000,1.100000,1.000000,0.000000,306.300000
25%,1958.000000,2.500000,44.000000,0.000000,4999.500000
50%,1973.000000,2.500000,63.000000,0.000000,7982.600000
75%,1998.000000,2.500000,79.700000,1.000000,12970.100000
max,2024.000000,4.600000,365.200000,1.000000,111399.400000


In [103]:
#On enlève les NA de la conso 
df_dpe_filtered = df_dpe_filtered.dropna(subset=["Conso_5_usages_é_finale"])
df_dpe_filtered.isnull().sum()

Etiquette_DPE                             0
Type_bâtiment                             0
Année_construction                   127491
Classe_inertie_bâtiment                2063
Hauteur_sous-plafond                      0
Surface_habitable_logement             2054
Type_énergie_principale_chauffage     10547
Isolation_toiture_(0/1)              126856
Code_INSEE_(BAN)                          0
Conso_5_usages_é_finale                   0
dtype: int64

In [104]:
#On enlève les "immeubles"
df_dpe_filtered = df_dpe_filtered[df_dpe_filtered['Type_bâtiment'] != 'immeuble']
df_dpe_filtered.shape

(311814, 10)

# Train / Test Split

In [105]:
#Séparation variables explicatives et variable cible 
X = df_dpe_filtered[var_explicatives]
y = df_dpe_filtered[target]

In [106]:
#Train / Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

X_train.shape

(218269, 9)

# Pipeline 

In [107]:
#Colonnes numériques et catégorielles / traitement spécifique pour isolation toiture 
num_features = ["Année_construction", "Hauteur_sous-plafond", "Surface_habitable_logement"]
iso_feature = ["Isolation_toiture_(0/1)"]
cat_features = ["Etiquette_DPE", "Type_bâtiment", "Classe_inertie_bâtiment", "Type_énergie_principale_chauffage", "Code_INSEE_(BAN)"]

In [108]:
#Pipeline pour les colonnes numériques 
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

In [109]:
#Pipeline pour la colonne isolation toiture 
iso_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Inconnue'))
])

In [110]:
#Pipeline pour les colonnes catégorielles 
# Étape 1 : Imputer les valeurs manquantes dans les colonnes catégorielles
categorical_imputer = SimpleImputer(strategy='most_frequent')
X_train_imputed = X_train.copy()  # Copie des données d'origine pour éviter les modifications
X_train_imputed[cat_features] = categorical_imputer.fit_transform(X_train[cat_features])

# Étape 2 : Appliquer le TargetEncoder sur les colonnes catégorielles déjà imputées
target_encoder = ce.TargetEncoder(cols=cat_features)
X_train_encoded = target_encoder.fit_transform(X_train_imputed, y_train)

# De même pour les données de test
X_test_imputed = X_test.copy()
X_test_imputed[cat_features] = categorical_imputer.transform(X_test[cat_features])
X_test_encoded = target_encoder.transform(X_test_imputed)

X_train_encoded.head()

,Etiquette_DPE,Type_bâtiment,Année_construction,Classe_inertie_bâtiment,Hauteur_sous-plafond,Surface_habitable_logement,Type_énergie_principale_chauffage,Isolation_toiture_(0/1),Code_INSEE_(BAN)
179545,19130.542152,8977.069565,1947.0,10953.227514,2.5,32.3,6045.263729,NaN,10332.399339
158139,6718.527208,8977.069565,1994.0,9449.518676,2.5,71.0,6045.263729,0.0,8357.360188
17952,10209.585069,8977.069565,1974.0,10953.227514,2.5,66.0,11052.888814,NaN,13130.418611
41861,10209.585069,20144.917377,1975.0,10953.227514,2.5,134.2,11703.464228,NaN,11427.587212
246221,6718.527208,8977.069565,1986.0,9449.518676,2.5,88.5,6045.263729,0.0,8037.725584


In [111]:
#Prepocessor 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features)
    ], remainder='passthrough'# Conserve les colonnes déjà encodées (catégorielles)
)

In [112]:
#Pipeline final avec la preprocessor et le modèle 
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regression', DecisionTreeRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=10))
])

In [113]:
# Pipeline final avec les données encodées et imputées
pipeline.fit(X_train_encoded, y_train)

# Sauvegarde du pipeline complet
joblib.dump(pipeline, 'pipeline_ml_regression.joblib')

['pipeline_ml_regression.joblib']